In [4]:
import os 

In [5]:
%pwd

'/home/omar/Desktop/End-to-end-Machine-Learning-Project-with-MLflow/research'

In [6]:
os.chdir("../")
%pwd

'/home/omar/Desktop/End-to-end-Machine-Learning-Project-with-MLflow'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [8]:
from src.ML.constants import *
from ML.utils.common import read_yaml, create_directories


In [9]:
class ConfigurationManager:
    def __init__(self,
                config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH,
                schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [10]:
import urllib.request as request
import zipfile
from ML import logger
from ML.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download with info : \n{headers}")
        else:
            logger.info(f"File already exist , size : {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip(self):
        """
        Extracts the zip file into the data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip:
            zip.extractall(unzip_path) 

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip()
except Exception as e:
    raise e


[2024-10-23 17:40:13,817: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-10-23 17:40:13,825: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-23 17:40:13,832: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-23 17:40:13,835: INFO: common: created directory at: artifacts]
[2024-10-23 17:40:13,839: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-23 17:40:15,284: INFO: 1392684224: artifacts/data_ingestion/data.zip download with info : 
Connection: close
Content-Length: 139819
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e43582789c59545b27df58df88f69b9f2426837bef2a2d35aa87f98f8eb22435"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F845:0E7C:21465:227CF:671922BE
Accept-Ranges: bytes
Date: Wed, 23 Oct 2024 16: